### **AI Research Scientit**
> **[``2024-07-24``]() $\rightarrow$ [``2025-06-26``]()**

```{Python}
- Tech Lead :: Gabriel L. S. Silva
```

In [5]:
#from HIERColect import OverColetor
from datetime import datetime
from ultralytics import YOLO

import matplotlib.pyplot as plt
import requests
import geocoder
import logging
import base64
import time
import json
import cv2


In [6]:
key = requests.get("https://raw.githubusercontent.com/gabrielluizone/FirstCode/refs/heads/main/conn.key").text.strip()
g = geocoder.ip('me')

In [ ]:
class OverColetor:
    def __init__(self, empresa_id, empresa_nome, coletor_id, coletor_descricao, coletor_localizacao,
                 camera_source=0, model_key="oil_spill", model_path="models/Vo1.pt", show_notebook=True,
                 capture_interval=15, confidence_threshold=0.7, ngrok_domain=""):
        """Inicializa o detector de manchas de óleo - versão corrigida"""
        
        # Configurações básicas
        self.EMPRESA_ID = empresa_id
        self.EMPRESA_NOME = empresa_nome
        self.COLETOR_ID = coletor_id
        self.MODELO_ID = model_key
        self.COLETOR_DESCRICAO = coletor_descricao
        self.COLETOR_LOCALIZACAO = coletor_localizacao
        
        # Configurações de operação
        self.CAPTURE_INTERVAL_SECONDS = capture_interval
        self.CONFIDENCE_THRESHOLD = confidence_threshold
        self.CAMERA_SOURCE = camera_source
        self.SHOW_NOTEBOOK = show_notebook
        
        # URLs do servidor - CORRIGIDO
        base_url = f"https://{ngrok_domain}.ngrok-free.app"
        self.OIL_DETECTION_URL = f"{base_url}/oil_detection"  # Endpoint correto
        self.SKF_ANALYSIS_URL = f"{base_url}/skf_analysis"
        self.HEALTH_URL = f"{base_url}/health"
        
        # Caminho do modelo
        self.MODEL_PATH = model_path
        
        # Configurar log básico
        self._setup_logger()
        
        # Testar conexão na inicialização
        self._test_server_connection()
        
    def _setup_logger(self):
        """Configura log básico"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger('OverColetor')
    
    def _test_server_connection(self):
        """Testa conexão com o servidor"""
        try:
            response = requests.get(self.HEALTH_URL, timeout=10)
            if response.ok:
                health_data = response.json()
                self.logger.info(f"✓ Servidor conectado: {health_data.get('status', 'unknown')}")
                return True
            else:
                self.logger.error(f"✗ Servidor não respondeu adequadamente: {response.status_code}")
                return False
        except Exception as e:
            self.logger.error(f"✗ Erro ao conectar com servidor: {str(e)}")
            return False
    
    def _encode_image_high_quality(self, image):
        """Codifica imagem em base64 mantendo alta qualidade"""
        try:
            # Usar PNG com compressão mínima para manter qualidade
            encode_params = [cv2.IMWRITE_PNG_COMPRESSION, 1]  # Compressão mínima (0-9, onde 0=sem compressão)
            success, buffer = cv2.imencode('.png', image, encode_params)
            
            if not success:
                raise ValueError("Falha ao codificar imagem")
            
            # Converter para base64
            image_b64 = base64.b64encode(buffer).decode('utf-8')
            
            # Log do tamanho para debug
            size_mb = len(image_b64) * 3/4 / (1024*1024)  # Aproximação do tamanho em MB
            #self.logger.debug(f"Imagem codificada: {size_mb:.2f}MB")
            
            return image_b64
            
        except Exception as e:
            self.logger.error(f"Erro ao codificar imagem: {str(e)}")
            return None
    
    def _send_data_to_server(self, original_image, processed_image, detection_data, metadata):
        """Envia dados para o servidor usando o endpoint correto"""
        try:
            # Codificar imagens em base64 com alta qualidade
            #self.logger.info("Codificando imagens...")
            imagem_original_b64 = self._encode_image_high_quality(original_image)
            imagem_processada_b64 = self._encode_image_high_quality(processed_image)
            
            if not imagem_original_b64 or not imagem_processada_b64:
                self.logger.error("Falha na codificação das imagens")
                return False
            
            # Criar timestamp ISO format
            now = datetime.now()
            timestamp_iso = now.isoformat()
            timestamp_file = now.strftime('%Y%m%d_%H%M%S_%f')[:-3]  # Formato para nome do arquivo
            detections_count = len(detection_data)
            
            # Nomes dos arquivos
            original_image_name = f"RAW-{self.EMPRESA_ID}-{self.COLETOR_ID}-{self.MODELO_ID}-{timestamp_file}-{detections_count}.png"
            processed_image_name = f"PRC-{self.EMPRESA_ID}-{self.COLETOR_ID}-{self.MODELO_ID}-{timestamp_file}-{detections_count}.png"
            
            # Preparar payload conforme esperado pelo servidor
            payload = {
                "imagem_original_base64": imagem_original_b64,
                "imagem_processada_base64": imagem_processada_b64,
                "name_imagem_original": original_image_name,
                "name_imagem_processada": processed_image_name,
                "empresa_info": {
                    "empresa_id": self.EMPRESA_ID,
                    "empresa_nome": self.EMPRESA_NOME,
                    "coletor_id": self.COLETOR_ID,
                    "modelo_id": self.MODELO_ID,
                    "coletor_descricao": self.COLETOR_DESCRICAO,
                    "localizacao": self.COLETOR_LOCALIZACAO
                },
                "detection_data": detection_data,
                "metadata": metadata,
                "timestamp": timestamp_iso,  # ISO format para timestamp
                "confidence_threshold": max([d['confidence'] for d in detection_data]) if detection_data else self.CONFIDENCE_THRESHOLD,
                "detections_count": detections_count
            }
            
            # Log do tamanho do payload
            payload_size_mb = len(json.dumps(payload).encode('utf-8')) / (1024*1024)
            self.logger.info(f"Enviando payload de {payload_size_mb:.2f}MB com {detections_count} detecções...")
            
            # Headers adequados
            headers = {
                'Content-Type': 'application/json',
                'Accept': 'application/json'
            }
            
            # Enviar dados com timeout maior para imagens grandes
            response = requests.post(
                self.OIL_DETECTION_URL, 
                json=payload, 
                headers=headers,
                timeout=60  # Timeout maior para imagens de alta qualidade
            )
            
            # Log da resposta completa para debug
            self.logger.info(f"Status da resposta: {response.status_code}")
            
            if response.ok:
                try:
                    result = response.json()
                    self.logger.debug(f"Resposta do servidor: {result}")
                    
                    # Verificar se foi processado com sucesso
                    if result.get("status") == "success" or result.get("return") == 1:
                        deteccao_id = result.get("deteccao_id")
                        self.logger.info(f"✓ Captura & Dados Enviados :: ID {deteccao_id}")
                        return True
                    else:
                        self.logger.warning(f"✗ Servidor não confirmou sucesso: {result}")
                        return False
                        
                except json.JSONDecodeError:
                    self.logger.error(f"✗ Resposta inválida do servidor: {response.text}")
                    return False
            else:
                self.logger.error(f"✗ Erro HTTP {response.status_code}: {response.text}")
                return False
                
        except requests.exceptions.Timeout:
            self.logger.error("✗ Timeout ao enviar dados (imagem muito grande?)")
            return False
        except requests.exceptions.ConnectionError:
            self.logger.error("✗ Erro de conexão com servidor")
            return False
        except Exception as e:
            self.logger.error(f"✗ Erro ao enviar dados: {str(e)}")
            return False
    
    def _detect_and_send(self):
        """Captura imagem, executa detecção e envia dados"""
        try:
            # Carregar modelo
            self.logger.debug("Carregando modelo YOLO...")
            model = YOLO(self.MODEL_PATH, verbose=False)
            
            # Capturar imagem da câmera
            self.logger.debug("Capturando imagem da câmera...")
            cap = cv2.VideoCapture(self.CAMERA_SOURCE)
            
            # Configurar resolução máxima da câmera (opcional)
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
            
            ret, frame = cap.read()
            if not ret:
                cap.release()
                raise ValueError("Não foi possível capturar imagem da câmera")
            
            original_image = frame.copy()
            cap.release()
            
            #self.logger.info(f"Imagem capturada: {original_image.shape[1]}x{original_image.shape[0]}")
            
            # Executar detecção
            self.logger.debug("Executando detecção YOLO...")
            results = model(original_image, conf=self.CONFIDENCE_THRESHOLD, verbose=False)
            
            # Processar resultados
            detections = []
            for result in results:
                for box in result.boxes:
                    detections.append({
                        'class': result.names[int(box.cls)],
                        'confidence': round(float(box.conf), 6),  # Mais precisão
                        'bbox': [round(coord, 2) for coord in box.xyxy[0].tolist()],
                        'bbox_normalized': [round(coord, 6) for coord in box.xywhn[0].tolist()]  # Coordenadas normalizadas
                    })
            
            #self.logger.info(f"Detecções encontradas: {len(detections)}")
            
            # Se não há detecções, apenas log (não enviar)
            if len(detections) == 0:
                #self.logger.info("Nenhuma detecção encontrada - não enviando")
                return False
            
            # Gerar imagem com detecções
            processed_image = results[0].plot()
            
            # Mostrar imagens se solicitado
            if self.SHOW_NOTEBOOK:
                self._display_results(original_image, processed_image, detections)
            
            # Criar metadados mais detalhados
            metadata = {
                'model_info': {
                    'model_path': self.MODEL_PATH,
                    'confidence_threshold': self.CONFIDENCE_THRESHOLD
                },
                'image_info': {
                    'original_shape': original_image.shape,
                    'channels': original_image.shape[2] if len(original_image.shape) > 2 else 1
                },
                'inference_speed': {
                    'preprocess_ms': round(results[0].speed['preprocess'], 4),
                    'inference_ms': round(results[0].speed['inference'], 4),
                    'postprocess_ms': round(results[0].speed['postprocess'], 4)
                },
                'detection_summary': {
                    'total_detections': len(detections),
                    'max_confidence': max([d['confidence'] for d in detections]) if detections else 0,
                    'classes_detected': list(set([d['class'] for d in detections]))
                },
                'timestamp_capture': datetime.now().isoformat()
            }
            
            # Enviar dados
            send_success = self._send_data_to_server(original_image, processed_image, detections, metadata)
            
            return send_success
            
        except Exception as e:
            self.logger.error(f"Erro na detecção: {str(e)}")
            return False
    
    def _display_results(self, original_image, processed_image, detections):
        """Exibe resultados da detecção"""
        try:
            original_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
            processed_rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
            
            plt.figure(figsize=(15, 6))
            
            plt.subplot(1, 2, 1)
            plt.imshow(original_rgb)
            plt.title('Imagem Original')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(processed_rgb)
            plt.title(f'Detecções ({len(detections)} encontradas)')
            plt.axis('off')
            
            # Adicionar lista de detecções
            detection_text = "\n".join([
                f"{d['class']}: {d['confidence']:.3f}" for d in detections[:5]  # Mostrar apenas 5 primeiras
            ])
            if len(detections) > 5:
                detection_text += f"\n... e mais {len(detections)-5}"
            
            plt.figtext(0.02, 0.02, detection_text, fontsize=8, verticalalignment='bottom')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            self.logger.warning(f"Erro ao exibir resultados: {str(e)}")
    
    def test_single_detection(self):
        """Executa uma única detecção para teste"""
        self.logger.info("=== TESTE DE DETECÇÃO ÚNICA ===")
        success = self._detect_and_send()
        if success:
            self.logger.info("✓ Teste concluído com sucesso!")
        else:
            self.logger.warning("✗ Teste falhou!")
        return success
    
    def run_continuous_monitoring(self):
        """Executa monitoramento contínuo"""
        #self.logger.info("=== INICIANDO MONITORAMENTO CONTÍNUO ===")
        self.logger.info(f"Empresa: {self.EMPRESA_NOME} ({self.EMPRESA_ID})")
        self.logger.info(f"Coletor: {self.COLETOR_DESCRICAO} ({self.COLETOR_ID})")
        self.logger.info(f"Intervalo: {self.CAPTURE_INTERVAL_SECONDS}s")
        self.logger.info(f"Confiança: {self.CONFIDENCE_THRESHOLD}")
        
        successful_sends = 0
        total_attempts = 0
        
        try:
            while True:
                total_attempts += 1
                #self.logger.info(f"\n--- Captura #{total_attempts} ---")
                
                success = self._detect_and_send()
                if success:
                    successful_sends += 1
                
                success_rate = (successful_sends / total_attempts) * 100
                #self.logger.info(f"Taxa de sucesso: {successful_sends}/{total_attempts} ({success_rate:.1f}%)")
                
                # Aguardar próxima captura
                if self.CAPTURE_INTERVAL_SECONDS > 0:
                    #self.logger.info(f"Aguardando {self.CAPTURE_INTERVAL_SECONDS}s...")
                    time.sleep(self.CAPTURE_INTERVAL_SECONDS)
                
        except KeyboardInterrupt:
            #self.logger.info(f"\n=== MONITORAMENTO INTERROMPIDO ===")
            self.logger.info(f"Total de tentativas: {total_attempts}")
            self.logger.info(f"Envios bem-sucedidos: {successful_sends}")
            self.logger.info(f"Taxa de sucesso final: {success_rate:.1f}%")
        except Exception as e:
            self.logger.error(f"Erro no monitoramento: {str(e)}")

In [ ]:
coletor = OverColetor(
    empresa_id='DEMO001',
    empresa_nome='Demonstracao',
    coletor_id='DMO005',
    coletor_descricao='Camera Teste - CENEP',
    coletor_localizacao=f'{g.latlng[0]}, {g.latlng[1]}' if g.latlng else  '-23.9931, -46.2564',
    camera_source=0,
    model_key="OilSpill",
    model_path="models/Vo1.pt",
    show_notebook=True,
    capture_interval=7,
    confidence_threshold=0.7,
    ngrok_domain=key
)
coletor.run_continuous_monitoring()